In [1]:
%pip install -r ../../../requirements.txt


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
import pandas as pd
import gzip
import json
import pandas as pd
from pymongo import MongoClient
from sklearn.metrics import roc_auc_score
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator


In [3]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)
def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')
purchasing_df = getDF('../../../Data/Video_Games_5.json.gz')
meta_datadf = getDF('../../../Data/meta_Video_Games.json.gz')

In [4]:
columns_to_drop = ['vote', 'style', 'image'] 
purchasing_df = purchasing_df.drop(columns=columns_to_drop)
positive_samples = purchasing_df.copy()
positive_samples['label'] = 1 
positive_samples['reviewTime'] = pd.to_datetime(positive_samples['reviewTime'])
positive_samples.head()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,label
0,5.0,True,2015-10-17,A1HP7NVNPFMA4N,0700026657,Ambrosia075,"This game is a bit hard to get the hang of, bu...",but when you do it's great.,1445040000,1
1,4.0,False,2015-07-27,A1JGAP0185YJI6,0700026657,travis,I played it a while but it was alright. The st...,"But in spite of that it was fun, I liked it",1437955200,1
2,3.0,True,2015-02-23,A1YJWEXHQBWK2B,0700026657,Vincent G. Mezera,ok game.,Three Stars,1424649600,1
3,2.0,True,2015-02-20,A2204E1TH211HT,0700026657,Grandma KR,"found the game a bit too complicated, not what...",Two Stars,1424390400,1
4,5.0,True,2014-12-25,A2RF5B5H74JLPE,0700026657,jon,"great game, I love it and have played it since...",love this game,1419465600,1


In [5]:
meta_datadf = meta_datadf.drop_duplicates(subset='asin', keep='first')
meta_datadf.reset_index(drop=True, inplace=True)
meta_datadf.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes,details
0,"[Video Games, PC, Games]",,[],,Reversi Sensory Challenger,[],,Fidelity Electronics,[],"[>#2,623,937 in Toys &amp; Games (See Top 100 ...",[],Toys &amp; Games,,,,0042000742,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
1,"[Video Games, Xbox 360, Games, </span></span><...",,[Brand new sealed!],,Medal of Honor: Warfighter - Includes Battlefi...,[B00PADROYW],,by\n \n EA Games,[],"[>#67,231 in Video Games (See Top 100 in Video...","[B0050SY5BM, B072NQJCW5, B000TI836G, B002SRSQ7...",Video Games,,,"\n\t\t\t\t\t\t\t\t\t\t\t\t<span class=""vertica...",0078764343,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
2,"[Video Games, Retro Gaming & Microconsoles, Su...",,[],,street fighter 2 II turbo super nintendo snes ...,[],,Nintendo,[],"[>#134,433 in Video Games (See Top 100 in Vide...",[],Video Games,,,$0.72,0276425316,[],[],NaN
3,"[Video Games, Xbox 360, Accessories, Controlle...",,[MAS's Pro Xbox 360 Stick (Perfect 360 Stick) ...,,Xbox 360 MAS STICK,[],,by\n \n MAS SYSTEMS,[Original PCB used from Xbox 360 Control Pad (...,"[>#105,263 in Video Games (See Top 100 in Vide...",[],Video Games,,,,0324411812,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN
4,"[Video Games, PC, Games, </span></span></span>...",,"[Phonics Alive! 3, The Speller teaches student...",,Phonics Alive! 3: The Speller,[],,by\n \n Advanced Software Pty. Ltd.,"[Grades 2-12, Spelling Program, Teaches Spelli...","[>#92,397 in Video Games (See Top 100 in Video...",[B000BCZ7U0],Video Games,,,,0439335310,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...,NaN


In [6]:
positive_samples = positive_samples.merge(meta_datadf, on='asin', how='inner')
positive_samples['price'] = pd.to_numeric(positive_samples['price'].str.replace(r'[^\d.]', '', regex=True), errors='coerce')
positive_samples['price'].isna().sum()

428894

In [7]:
users_to_instances = {} 
for index, row in positive_samples.iterrows(): 
    if row['reviewerID'] not in users_to_instances:
        users_to_instances[row['reviewerID']] = 0
    users_to_instances[row['reviewerID']] += 1

In [8]:
positive_samples = positive_samples[positive_samples['reviewerID'].map(users_to_instances) >= 10]

In [9]:
item_to_instances = {} 
for index, row in positive_samples.iterrows(): 
    if row['asin'] not in item_to_instances:
        item_to_instances[row['asin']] = 0
    item_to_instances[row['asin']] += 1

In [10]:
positive_samples = positive_samples[positive_samples['asin'].map(item_to_instances) >= 10]

In [11]:
positive_samples.shape

(186198, 28)

In [12]:
columns_to_drop = ['imageURL', 'imageURLHighRes', 'reviewerName', 'reviewText', 
                   'unixReviewTime', 'date']
positive_samples = positive_samples.drop(columns=columns_to_drop, axis=1)
positive_samples = positive_samples.reset_index(drop=True)

In [13]:
negative_samples = pd.DataFrame(columns=positive_samples.columns)

In [14]:
client = MongoClient('mongodb://localhost:27017/')
db = client['video_games_db']
price_collection = db['keepa']
query = {"df_NEW": {"$exists": True}}
document_count = price_collection.count_documents(query)
projection = {"df_NEW": 1, "asin": 1, "_id": 0}
documents = price_collection.find(query, projection)
price_df = pd.DataFrame(documents) 

In [15]:
items_considered = set() 
for index, row in positive_samples.iterrows(): 
    items_considered.add(row["asin"])

In [16]:
asin_to_price = {}
for index, row in price_df.iterrows():
    asin_to_price[row["asin"]] = row["df_NEW"]

In [17]:
asin_to_positive = {} 
for index, row in positive_samples.iterrows(): 
    asin_to_positive[row['asin']] = index

In [18]:
asin_to_meta = {}
for index, row in meta_datadf.iterrows(): 
    asin_to_meta[row["asin"]] = index

In [19]:
person_to_purchases = {} 
for index, row in positive_samples.iterrows(): 
    if row['reviewerID'] not in person_to_purchases:
        person_to_purchases[row['reviewerID']] = set()
    person_to_purchases[row['reviewerID']].add(row['asin'])

In [20]:
person_to_not_purchased = {} 
all_asins = set()
for index, row in positive_samples.iterrows(): 
    all_asins.add(row['asin'])

In [21]:
for index, row in positive_samples.iterrows():
    if row['reviewerID'] not in person_to_not_purchased:
        person_to_not_purchased[row['reviewerID']] = all_asins.copy()
    if row['asin'] in person_to_not_purchased[row['reviewerID']]:
        person_to_not_purchased[row['reviewerID']].remove(row['asin'])

In [22]:
len(person_to_not_purchased[positive_samples.iloc[0]['reviewerID']])

6150

In [23]:
len(person_to_purchases[positive_samples.iloc[0]['reviewerID']])

9

In [24]:
len(all_asins)

6159

In [25]:
print(positive_samples.shape)

(186198, 22)


In [26]:
from tqdm import tqdm
tqdm.pandas()
import random
dfs_to_concat = [] 
for index, row in tqdm(positive_samples.iterrows(), total=positive_samples.shape[0], desc="Processing rows"):
    current_reviewer = row['reviewerID'] 
    current_date = pd.to_datetime(row['reviewTime'])
    for i in range(10): 
        negative_sample_asin = list(person_to_not_purchased[current_reviewer]).pop(random.randint(0, len(person_to_not_purchased[current_reviewer]) - 1))
        new_row = positive_samples.iloc[asin_to_positive[negative_sample_asin]].to_dict()
        new_row['label'] = 0
        new_row['reviewTime'] = current_date
        new_row['reviewerID'] = current_reviewer
        dfs_to_concat.append(new_row)

Processing rows:  67%|██████▋   | 124214/186198 [07:13<03:36, 286.43it/s]


KeyboardInterrupt: 

In [ ]:
negative_samples = pd.DataFrame(dfs_to_concat)
print(negative_samples.shape)
print(positive_samples.shape)

(1861980, 22)
(186198, 22)


In [ ]:
combined_samples = pd.concat([positive_samples, negative_samples], ignore_index=True)

In [ ]:
print(combined_samples.head())

   overall  verified reviewTime      reviewerID        asin  \
0      5.0      True 2011-08-14   AN3YYDZAS3O1Y  0700099867   
1      5.0     False 2011-06-18  A15PIAQT55GNCA  0700099867   
2      4.0      True 2011-06-14  A361M14PU2GUEG  0700099867   
3      5.0     False 2011-06-13  A2LQCBLLJVVR5T  0700099867   
4      1.0      True 2014-06-30  A248LSBZT4P38V  0700099867   

                                             summary  label  \
0        A step up from Dirt 2 and that is terrific!      1   
1                      this games is amazing!!!!!!!!      1   
2                                             DIRT 3      1   
3              BEST GRAPHICS OF ANY GAME SO FAR !!!!      1   
4  It might have been a good game, but I never fo...      1   

                   category tech1  \
0  [Video Games, PC, Games]         
1  [Video Games, PC, Games]         
2  [Video Games, PC, Games]         
3  [Video Games, PC, Games]         
4  [Video Games, PC, Games]         

                   

In [ ]:
def binary_search_closest(arr, target, compare):
    low, high = 0, len(arr) - 1
    closest = None
    closest_diff = float('inf')
    while low <= high:
        mid = (low + high) // 2
        result = compare(arr[mid], target)
        if result == 0:
            return arr[mid]
        elif result < 0:
            low = mid + 1
        else:
            high = mid - 1
        diff = abs(result)
        if diff < closest_diff:
            closest_diff = diff
            closest = arr[mid]
    return closest

In [ ]:
from tqdm import tqdm
tqdm.pandas()
removed_rows = set()
for index, row in tqdm(combined_samples.iterrows(), total=combined_samples.shape[0], desc="Processing rows"):
    asin = row["asin"]
    current_date = row["reviewTime"]
    if asin in asin_to_price:
        keys_to_be_removed = []
        for key in asin_to_price[asin]:
            if np.isnan(asin_to_price[asin][key]):
                keys_to_be_removed.append(key)
        for key in keys_to_be_removed:
            del asin_to_price[asin][key]
        price_list = list(asin_to_price[asin].keys())
        if len(price_list) == 0:
            removed_rows.add(index)
        else: 
            closest_date = binary_search_closest(price_list, current_date, lambda x, y: (pd.to_datetime(x) - y).total_seconds())
            closest_price = asin_to_price[asin][closest_date]
            combined_samples.at[index, "price"] = closest_price
    else:
        removed_rows.add(index)
print(len(removed_rows))                                                                                                                                                        

Processing rows: 100%|██████████| 2048178/2048178 [48:51<00:00, 698.75it/s] 

278930


In [ ]:
combined_samples = combined_samples.drop(removed_rows)
combined_samples.reset_index(drop=True, inplace=True)

In [ ]:
print(combined_samples.shape)

(1769248, 22)


In [ ]:
combined_samples = combined_samples.dropna(subset=['price'])
combined_samples.reset_index(drop=True, inplace=True)

In [ ]:
print(combined_samples.shape)

(1769248, 22)


In [ ]:
asin_to_desc = {}
for index, row in combined_samples.iterrows(): 
    asin_to_desc[row['asin']] = row['description']

In [ ]:
combined_samples.colunns

In [5]:
#Read combined_samples from csv 
combined_samples = pd.read_csv("combined_samples.csv")

/var/folders/hq/878shj3x61n3xw850_m1scv00000gn/T/ipykernel_11972/3543859026.py:2: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_samples = pd.read_csv("combined_samples.csv")


In [6]:
combined_samples = combined_samples.sample(frac=1).reset_index(drop=True)

In [7]:
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()
brand_encoder = LabelEncoder()
combined_samples['user_id_encoded'] = user_encoder.fit_transform(combined_samples['reviewerID'])
combined_samples['item_id_encoded'] = item_encoder.fit_transform(combined_samples['asin'])
combined_samples['brand_id_encoded'] = brand_encoder.fit_transform(combined_samples['brand'])
combined_samples = combined_samples[['user_id_encoded', 'item_id_encoded', 'brand_id_encoded', 'price', 'description', 'label']]
num_users =  combined_samples["user_id_encoded"].nunique()
num_items = combined_samples["item_id_encoded"].nunique()
num_brands =  combined_samples["brand_id_encoded"].nunique()

In [8]:
X = combined_samples[['user_id_encoded', 'item_id_encoded', 'brand_id_encoded', 'price', 'description']].values
y = combined_samples['label'].values

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

positive_indices = np.where(y_train == 1)[0]
X_train_positive = X_train[positive_indices]
y_train_positive = y_train[positive_indices]
positive_indices_test = np.where(y_test == 1)[0]
X_test_positive = X_test[positive_indices_test]
y_test_positive = y_test[positive_indices_test]

for i in range(10):
    X_train = np.concatenate((X_train, X_train_positive))
    y_train = np.concatenate((y_train, y_train_positive))
    X_test = np.concatenate((X_test, X_test_positive))
    y_test = np.concatenate((y_test, y_test_positive))

permutation = np.random.permutation(len(X_train))
X_train_shuffled = X_train[permutation]
y_train_shuffled = y_train[permutation]


X_train_user = X_train[:, 0].astype(np.int32)
X_train_item = X_train[:,1].astype(np.int32)
X_train_brand = X_train[:,2].astype(np.int32)
X_train_price = X_train[:,3].astype(np.float32)
X_train_description = X_train[:, 4].astype(np.str_)

X_test_user = X_test[:, 0].astype(np.int32)
X_test_item = X_test[:, 1].astype(np.int32)
X_test_brand = X_test[:, 2].astype(np.int32)
X_test_price = X_test[:, 3].astype(np.float32)
X_test_description = X_test[:, 4].astype(np.str_)

y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

: 

In [ ]:
tokenizer = get_tokenizer("basic_english")
def yield_tokens(data_iter):
    for text in data_iter:
        yield tokenizer(text)
vocab = build_vocab_from_iterator(yield_tokens(X_train_description), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])
def text_pipeline(text):
    return [vocab[token] for token in tokenizer(text)]

X_train_description_processed = [text_pipeline(text) for text in X_train_description]
X_test_description_processed = [text_pipeline(text) for text in X_test_description]
max_length = max(len(desc) for desc in X_train_description_processed + X_test_description_processed)

def pad_sequence(seq):
    return seq + [vocab["<unk>"]] * (max_length - len(seq))

X_train_description_padded = torch.tensor([pad_sequence(desc) for desc in X_train_description_processed], dtype=torch.long)
X_test_description_padded = torch.tensor([pad_sequence(desc) for desc in X_test_description_processed], dtype=torch.long)

In [ ]:
X_train_user = torch.tensor(X_train_user, dtype=torch.int32)
X_train_item = torch.tensor(X_train_item, dtype=torch.int32)
X_train_brand = torch.tensor(X_train_brand, dtype=torch.int32)
X_train_price = torch.tensor(X_train_price, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)

X_test_user = torch.tensor(X_test_user, dtype=torch.int32)
X_test_item = torch.tensor(X_test_item, dtype=torch.int32)
X_test_brand = torch.tensor(X_test_brand, dtype=torch.int32)
X_test_price = torch.tensor(X_test_price, dtype=torch.float32)

y_test = torch.tensor(y_test, dtype=torch.float32)


train_dataset = TensorDataset(X_train_user, X_train_item, X_train_brand, X_train_price, X_train_description_padded, y_train)
test_dataset = TensorDataset(X_test_user, X_test_item, X_test_brand, X_test_price, X_test_description_padded, y_test)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)


class RecommendationModel(nn.Module):
    def __init__(self, num_users, num_items, num_brands, vocab_size, embedding_size, description_embedding_size):
        super(RecommendationModel, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_size)
        self.item_embedding = nn.Embedding(num_items, embedding_size)
        self.brand_embedding = nn.Embedding(num_brands, embedding_size)
        self.description_embedding = nn.Embedding(vocab_size, description_embedding_size)
        self.fc1 = nn.Linear(embedding_size * 3 + 1 + description_embedding_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, user, item, brand, price, description):
        user_vec = self.user_embedding(user)
        item_vec = self.item_embedding(item)
        brand_vec = self.brand_embedding(brand)
        desc_vec = self.description_embedding(description).mean(dim=1)  
        x = torch.cat([user_vec, item_vec, brand_vec, price.unsqueeze(1), desc_vec], dim=1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=0.5)
        x = torch.sigmoid(self.fc3(x))
        return x

torch.Size([2737380])
torch.Size([2737380, 1])


/var/folders/hq/878shj3x61n3xw850_m1scv00000gn/T/ipykernel_63778/3158063745.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_user = torch.tensor(X_train_user, dtype=torch.int32)
/var/folders/hq/878shj3x61n3xw850_m1scv00000gn/T/ipykernel_63778/3158063745.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_item = torch.tensor(X_train_item, dtype=torch.int32)
/var/folders/hq/878shj3x61n3xw850_m1scv00000gn/T/ipykernel_63778/3158063745.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_brand = torch.tensor(X_train_b

In [ ]:
vocab_size = len(vocab)
model = RecommendationModel(num_users=num_users, num_items=num_items, num_brands=num_brands, vocab_size=vocab_size, embedding_size=32, description_embedding_size=100)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
def calculate_accuracy(outputs, labels):
    predictions = (torch.sigmoid(outputs) > 0.5).float()
    correct = (predictions == labels).float().sum()
    return correct / len(labels)

for epoch in range(50):
    model.train()
    all_outputs = []
    all_labels = []
    total_loss = 0
    total_accuracy = 0
    num_batches = 0
    for users, items, brands, prices, descriptions, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(users, items, brands, prices, descriptions)
        loss = criterion(outputs.squeeze(), labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        total_accuracy += calculate_accuracy(outputs.squeeze(), labels)
        num_batches += 1
        all_outputs.extend(outputs.detach().cpu().numpy())
        all_labels.extend(labels.detach().cpu().numpy())

    train_auc = roc_auc_score(all_labels, all_outputs)
    train_loss = total_loss / num_batches
    train_accuracy = total_accuracy / num_batches
    
    model.eval()
    total_val_loss = 0
    total_val_accuracy = 0
    num_val_batches = 0
    all_val_outputs = []
    all_val_labels = []
    
    with torch.no_grad():
        for users, items, brands, prices, descriptions, labels in test_loader:
            outputs = model(users, items, brands, prices, descriptions)
            val_loss = criterion(outputs.squeeze(), labels)
            total_val_loss += val_loss.item()
            total_val_accuracy += calculate_accuracy(outputs.squeeze(), labels)
            num_val_batches += 1
            all_val_outputs.extend(outputs.detach().cpu().numpy())
            all_val_labels.extend(labels.detach().cpu().numpy())
        
    val_loss = total_val_loss / num_val_batches
    val_accuracy = total_val_accuracy / num_val_batches
    model.train()
    val_auc = roc_auc_score(all_val_labels, all_val_outputs)
    print(f"Epoch {epoch+1}/{50}")
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
    print(f"Train AUC: {train_auc:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")
    print(f"Val AUC: {val_auc:.4f}")
    print("--------------------")

torch.Size([512]) torch.Size([512]) torch.Size([512]) torch.Size([512]) torch.Size([512, 1]) torch.Size([512])
torch.Size([512]) torch.Size([512]) torch.Size([512]) torch.Size([512]) torch.Size([512, 1]) torch.Size([512])
torch.Size([512]) torch.Size([512]) torch.Size([512]) torch.Size([512]) torch.Size([512, 1]) torch.Size([512])
torch.Size([512]) torch.Size([512]) torch.Size([512]) torch.Size([512]) torch.Size([512, 1]) torch.Size([512])
torch.Size([512]) torch.Size([512]) torch.Size([512]) torch.Size([512]) torch.Size([512, 1]) torch.Size([512])
torch.Size([512]) torch.Size([512]) torch.Size([512]) torch.Size([512]) torch.Size([512, 1]) torch.Size([512])
torch.Size([512]) torch.Size([512]) torch.Size([512]) torch.Size([512]) torch.Size([512, 1]) torch.Size([512])
torch.Size([512]) torch.Size([512]) torch.Size([512]) torch.Size([512]) torch.Size([512, 1]) torch.Size([512])
torch.Size([512]) torch.Size([512]) torch.Size([512]) torch.Size([512]) torch.Size([512, 1]) torch.Size([512])
t

KeyboardInterrupt: 